In [1]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor 

from seaborn import pairplot

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
# Инициализируем все известные генераторы случаынйх чисел / Setting all known random seeds
my_code = "Johnson"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

os.environ['PYTHONHASHSEED']=str(my_seed)

random.seed(my_seed)

np.random.seed(my_seed)

In [3]:
# Читаем данные из файла
train_data = pd.read_csv("datasets/rus_stocks_funds_train.csv")

In [4]:
train_data.head()

,Total issued ordinary shares,Number of shares available for sale,Number of employees,Number of shareholders,Price/Profit,Debt/Assets
0,7.210000e+09,2.870000e+09,110809.6875,3190.000000,9.870000,0.27
1,1.060000e+10,1.130000e+09,318000.0000,28230.000000,13.910000,0.30
2,2.249000e+10,1.027000e+10,329570.0000,61044.948718,4.080000,0.25
3,4.995200e+08,1.964200e+08,59380.0000,81.000000,2.910000,1.31
4,9.520000e+09,1.309960e+11,110809.6875,61044.948718,15.525191,0.35


In [5]:
#pairplot(train_data)

In [6]:
# Определим размер валидационной выборки
val_size = round(0.2*len(train_data))
print(val_size)

66


In [7]:
# Создадим обучающую и валидационную выборки
random_state = my_seed
train, val = train_test_split(train_data, test_size=val_size, random_state=random_state)
print(len(train), len(val))

263 66


In [8]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
y_column = 'Debt/Assets'
x_columns = [e for e in train_data.columns if e != y_column]

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), x_columns)], remainder='passthrough')

ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['Total issued ordinary shares',
                                  'Number of shares available for sale',
                                  'Number of employees',
                                  'Number of shareholders', 'Price/Profit'])])

In [9]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_val = pd.DataFrame(ct.transform(val))

In [10]:
# Устанавливаем названия столбцов
column_names = list(x_columns) + [y_column]
sc_train.columns = column_names
sc_val.columns = column_names

In [11]:
sc_train

,Total issued ordinary shares,Number of shares available for sale,Number of employees,Number of shareholders,Price/Profit,Debt/Assets
0,0.000019,0.023842,0.231213,0.000103,0.051813,0.12
1,0.000691,0.049084,0.231213,0.149645,0.029332,0.21
2,0.000175,0.049084,0.231213,0.149645,0.069377,0.35
3,0.005863,0.027575,0.231213,0.692601,0.010846,0.21
4,0.000444,0.024265,0.118138,0.000000,0.032932,0.53
...,...,...,...,...,...,...
258,0.001269,0.030674,0.231213,0.076527,0.063121,0.00
259,0.000444,0.024098,0.119557,0.000000,0.009309,0.51
260,0.000444,0.024101,0.116422,0.000000,0.016642,0.61
261,0.000210,0.049084,0.231213,0.149645,0.092869,0.32


In [12]:
# Отберем необходимые параметры
x_train = sc_train[x_columns]
x_val = sc_val[x_columns]

y_train = (sc_train[y_column].values).flatten()
y_val = (sc_val[y_column].values).flatten()

In [13]:
# Создадим простую модель логистической регрессии
model = TransformedTargetRegressor(regressor=LinearRegression(), transformer=MinMaxScaler())

In [14]:
# Обучим модель
model.fit(x_train, y_train)

TransformedTargetRegressor(regressor=LinearRegression(),
                           transformer=MinMaxScaler())

In [15]:
# Проверим работу обученной модели на валидационной выборке
pred_val = model.predict(x_val)
mse = mean_squared_error(y_val, pred_val)
print(mse)

0.06225053605015133


In [16]:
pred_val

array([0.42945689, 0.33661569, 0.33955468, 0.22537816, 0.40119091,
       0.33981055, 0.36686139, 0.22442996, 0.33467435, 0.33963302,
       0.37839735, 0.33820541, 0.38418762, 0.35913726, 0.33941317,
       0.33912358, 0.34013135, 0.07428396, 0.33231989, 0.38300217,
       0.07423179, 0.34017186, 0.42921183, 0.33819233, 0.33837921,
       0.35391013, 0.38080223, 0.33805228, 0.38356802, 0.38442347,
       0.41097829, 0.30138249, 0.35423524, 0.33788552, 0.40868469,
       0.38379401, 0.2253135 , 0.33219555, 0.33936806, 0.3383364 ,
       0.33973282, 0.33980089, 0.38319811, 0.13114521, 0.38065417,
       0.301047  , 0.41359864, 0.383085  , 0.25502962, 0.13221364,
       0.38905076, 0.32075881, 0.22438841, 0.45203842, 0.33970675,
       0.25516739, 0.33897928, 0.20140158, 0.36207406, 0.37436895,
       0.33851902, 0.33981469, 0.37853492, 0.38253872, 0.33712503,
       0.33792977])

In [17]:
test = pd.read_csv("datasets/rus_stocks_funds_test.csv")

In [18]:
test[y_column] = 0.0

In [19]:
test

,Total issued ordinary shares,Number of shares available for sale,Number of employees,Number of shareholders,Price/Profit,Debt/Assets
0,3.644700e+11,1.017600e+11,5840.0000,306350.000000,7.860000,0.0
1,6.968900e+08,4.339700e+08,110809.6875,39210.000000,5.670000,0.0
2,1.580000e+07,3.880000e+06,110809.6875,29.000000,7.060000,0.0
3,1.295000e+08,3.573000e+07,110809.6875,25.000000,14.940000,0.0
4,2.400000e+09,1.309960e+11,110809.6875,61044.948718,15.525191,0.0
...,...,...,...,...,...,...
77,1.528600e+08,1.309960e+11,110809.6875,61044.948718,6.270000,0.0
78,1.295000e+08,2.634000e+07,110809.6875,25.000000,15.525191,0.0
79,2.246000e+10,0.000000e+00,281550.0000,61044.948718,6.340000,0.0
80,1.295000e+08,3.371000e+07,110809.6875,25.000000,6.320000,0.0


In [20]:
sc_test = pd.DataFrame(ct.transform(test))
sc_test.columns = column_names

In [21]:
x_test = sc_test[x_columns]

In [22]:
test[y_column] = model.predict(x_test)

In [23]:
test.head()

,Total issued ordinary shares,Number of shares available for sale,Number of employees,Number of shareholders,Price/Profit,Debt/Assets
0,3.644700e+11,1.017600e+11,5840.0000,306350.000000,7.860000,0.219138
1,6.968900e+08,4.339700e+08,110809.6875,39210.000000,5.670000,0.355237
2,1.580000e+07,3.880000e+06,110809.6875,29.000000,7.060000,0.384219
3,1.295000e+08,3.573000e+07,110809.6875,25.000000,14.940000,0.382734
4,2.400000e+09,1.309960e+11,110809.6875,61044.948718,15.525191,0.338192


In [24]:
test.to_csv('task5_results/rus_stocks_funds_result.csv', index=False)